# 🧠 Day 3 – SQL via Python: NYC School Data Exploration
In this notebook, you'll connect to a PostgreSQL database and execute SQL queries to explore NYC school data.

## 🔌 Step 1: Import Libraries

In [26]:
import pandas as pd
import psycopg2

## 🔐 Step 2: Connect to the Database

In [27]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

## 🔍 Step 3: Run a Test Query

In [28]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()


/var/folders/sw/1gbgjnyn4_n7jh1f2rlvz1980000gn/T/ipykernel_5465/3773410935.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


## ✅ Task Queries Below

In [29]:
# Example: Count schools by borough
query = """
SELECT borough, COUNT(*) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_result = pd.read_sql(query, conn)
df_result

/var/folders/sw/1gbgjnyn4_n7jh1f2rlvz1980000gn/T/ipykernel_5465/3450537001.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,school_count
0,Brooklyn,121
1,Queens,80
2,Staten Island,10
3,Manhattan,106
4,Bronx,118


Column and Data Type Overview: `school_demographics`

In [22]:
#column and data type overview: `school_demographics`
pd.read_sql("""
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'nyc_schools'
  AND table_name  = 'school_demographics'
ORDER BY ordinal_position;
""", conn)




/var/folders/sw/1gbgjnyn4_n7jh1f2rlvz1980000gn/T/ipykernel_5465/605893319.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,column_name,data_type
0,dbn,character varying
1,Name,character varying
2,schoolyear,integer
3,fl_percent,character varying
4,frl_percent,real
5,total_enrollment,integer
6,prek,character varying
7,k,character varying
8,grade1,character varying
9,grade2,character varying


Average ELL % per Borough

In [30]:
q_ell_by_borough = """
WITH latest AS (
  SELECT DISTINCT ON (sd.dbn)
    sd.dbn,
    sd.ell_percent,
    sd.schoolyear
  FROM nyc_schools.school_demographics sd
  ORDER BY sd.dbn, sd.schoolyear DESC
)
SELECT
  d.borough,
  AVG(sd.ell_percent) AS avg_ell_percent
FROM latest sd
JOIN nyc_schools.high_school_directory d
  ON TRIM(sd.dbn) = TRIM(d.dbn)
GROUP BY d.borough
ORDER BY d.borough;
"""
df_ell_by_borough = pd.read_sql(q_ell_by_borough, conn)
df_ell_by_borough


/var/folders/sw/1gbgjnyn4_n7jh1f2rlvz1980000gn/T/ipykernel_5465/826438265.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell_by_borough = pd.read_sql(q_ell_by_borough, conn)


,borough,avg_ell_percent
0,Manhattan,8.814286


Checking Borough Coverage in `school_demographics` as only Manhattan appears

In [31]:
pd.read_sql("""
SELECT DISTINCT d.borough
FROM nyc_schools.school_demographics sd
JOIN nyc_schools.high_school_directory d
  ON TRIM(sd.dbn) = TRIM(d.dbn)
""", conn)


/var/folders/sw/1gbgjnyn4_n7jh1f2rlvz1980000gn/T/ipykernel_5465/1313751010.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,borough
0,Manhattan


Top 3 SPED Schools per Borough

In [24]:
q_top_sped_by_borough = """
WITH latest AS (
  SELECT DISTINCT ON (sd.dbn)
    sd.dbn,
    sd.sped_percent,
    sd.schoolyear
  FROM nyc_schools.school_demographics sd
  ORDER BY sd.dbn, sd.schoolyear DESC
),
ranked AS (
  SELECT
    d.borough,
    d.school_name,
    latest.dbn,
    latest.sped_percent,
    ROW_NUMBER() OVER (
      PARTITION BY d.borough
      ORDER BY latest.sped_percent DESC
    ) AS rn
  FROM latest
  JOIN nyc_schools.high_school_directory d
    ON TRIM(latest.dbn) = TRIM(d.dbn)
)
SELECT borough, school_name, dbn, sped_percent
FROM ranked
WHERE rn <= 3
ORDER BY borough, sped_percent DESC;
"""
df_top_sped_by_borough = pd.read_sql(q_top_sped_by_borough, conn)
df_top_sped_by_borough


/var/folders/sw/1gbgjnyn4_n7jh1f2rlvz1980000gn/T/ipykernel_5465/2775005773.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top_sped_by_borough = pd.read_sql(q_top_sped_by_borough, conn)


,borough,school_name,dbn,sped_percent
0,Manhattan,East Side Community School,01M450,26.4
1,Manhattan,Marta Valle High School,01M509,25.9
2,Manhattan,Henry Street School for International Studies,01M292,24.9


## 🧠 Insights

Summary of Observations and Findings

1. School Distribution (Task 1) 
• Brooklyn has the highest number of schools (121), followed by the Bronx (118) and Manhattan (106).  
• Staten Island has the fewest (10).  
• This shows a clear concentration of schools in Brooklyn and the Bronx.

2. Average % of English Language Learners (ELL) per Borough**  
• The query was designed to use only the most recent `schoolyear` for each school (`DISTINCT ON` by DBN).  
• Data limitation: In the current training database, `school_demographics` contains records only for Manhattan schools.  
• As a result, the average ELL percentage could only be calculated for Manhattan: 8.81%.

3. Top 3 Schools per Borough by Special Education % (SPED)**  
• The ranking was based on the latest `schoolyear` data per school.  
• Due to the same data limitation, results are available only for Manhattan.  
• The top 3 SPED schools in Manhattan have SPED percentages between 24.9% and 26.4%.

Overall Insights: 
• Brooklyn and the Bronx have the highest school counts, but no recent demographic data was available in this dataset for them.  
• The lack of borough coverage in `school_demographics` limits demographic analysis for this exercise.  

